In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [2]:
df = pd.read_csv("database_mscc.csv")

In [3]:
#df_for_model[0] = df_for_model[0].str.replace('[','')

In [4]:
df.head(5)

,Unnamed: 0,y,0,1,2,3,4,5,6,7,...,219,220,221,222,223,224,225,226,227,228
0,0,female_surprised,-29.824322,-30.296280,-30.341100,-30.838333,-31.067204,-30.690607,-29.505543,-30.370670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,female_fearful,-31.363651,-31.014317,-31.938639,-33.056618,-30.624622,-29.960500,-32.259970,-34.145702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,female_disgust,-31.545818,-31.020504,-30.576956,-31.558160,-31.198029,-30.364178,-29.075739,-30.090372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,female_surprised,-34.585396,-31.664700,-30.524185,-30.788443,-31.743572,-32.151950,-31.750254,-30.014618,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,female_disgust,-30.627995,-31.362064,-31.129383,-32.042410,-33.471330,-34.445076,-33.192543,-31.554546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.drop(columns = "Unnamed: 0", inplace=True)

In [6]:
df.tail()

,y,0,1,2,3,4,5,6,7,8,...,219,220,221,222,223,224,225,226,227,228
1435,female_disgust,-29.268341,-29.184180,-30.172647,-30.977152,-31.838312,-32.326775,-31.773617,-30.989521,-29.947556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1436,female_fearful,-30.255762,-29.088932,-28.198680,-27.928244,-27.569020,-27.578817,-27.733551,-27.521640,-28.491644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1437,female_surprised,-31.644619,-28.903180,-28.716227,-29.596594,-30.896626,-30.909430,-29.661102,-30.447230,-32.236750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438,female_surprised,-31.467510,-30.668491,-30.403183,-28.884400,-28.901764,-30.096973,-30.307621,-30.522541,-31.195232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,female_disgust,-34.253716,-32.424217,-31.304874,-30.410060,-30.121990,-29.580973,-30.290380,-31.769410,-33.591330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for row in np.arange(0,df.shape[0]):
    df["y"][row] = df["y"][row][7:]

/var/folders/7d/cjq_vxdx7vqfc6jkyf5tpx4r0000gn/T/ipykernel_8424/4108488883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["y"][row] = df["y"][row][7:]


In [8]:
df.isna().sum()

y         0
0         0
1         0
2         0
3         0
       ... 
224    1439
225    1439
226    1439
227    1439
228    1440
Length: 230, dtype: int64

In [9]:
df = df.fillna(0)

In [10]:
df["y"].drop_duplicates()

0     surprised
1       fearful
2       disgust
14        happy
15         calm
18      neutral
34          sad
35        angry
Name: y, dtype: object

In [11]:
y = df["y"]
X = df.drop(columns=["y"])

In [12]:
X

,0,1,2,3,4,5,6,7,8,9,...,219,220,221,222,223,224,225,226,227,228
0,-29.824322,-30.296280,-30.341100,-30.838333,-31.067204,-30.690607,-29.505543,-30.370670,-30.751425,-30.751398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-31.363651,-31.014317,-31.938639,-33.056618,-30.624622,-29.960500,-32.259970,-34.145702,-34.189300,-33.045395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-31.545818,-31.020504,-30.576956,-31.558160,-31.198029,-30.364178,-29.075739,-30.090372,-29.831460,-30.690586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-34.585396,-31.664700,-30.524185,-30.788443,-31.743572,-32.151950,-31.750254,-30.014618,-30.377722,-31.105500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-30.627995,-31.362064,-31.129383,-32.042410,-33.471330,-34.445076,-33.192543,-31.554546,-31.159200,-31.269037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,-29.268341,-29.184180,-30.172647,-30.977152,-31.838312,-32.326775,-31.773617,-30.989521,-29.947556,-28.873627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1436,-30.255762,-29.088932,-28.198680,-27.928244,-27.569020,-27.578817,-27.733551,-27.521640,-28.491644,-29.975315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1437,-31.644619,-28.903180,-28.716227,-29.596594,-30.896626,-30.909430,-29.661102,-30.447230,-32.236750,-31.757404,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1438,-31.467510,-30.668491,-30.403183,-28.884400,-28.901764,-30.096973,-30.307621,-30.522541,-31.195232,-31.903524,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [14]:
lb = LabelEncoder()
y_train_cat = np_utils.to_categorical(lb.fit_transform(y_train))
y_test_cat = np_utils.to_categorical(lb.fit_transform(y_test))

In [15]:
scaler = MinMaxScaler()

In [16]:
#scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
X_train_scaled_array = np.array(X_train_scaled)
X_test_scaled_array = np.array(X_test_scaled)
y_train_array = np.array(y_train_cat)
y_test_array = np.array(y_test_cat)

In [18]:
X_train_expanded_dims = np.expand_dims(X_train_scaled_array, axis=2)
X_test_expanded_dims = np.expand_dims(X_test_scaled_array, axis=2)

In [19]:
"""from tensorflow import keras
model = keras.Sequential([

        # input layer
        keras.layers.Flatten(input_shape=(X_train_expanded_dims.shape[1],1)),

        # 1st dense layer
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

        # 2nd dense layer
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

        # 3rd dense layer
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

        # output layer
        keras.layers.Dense(8, activation='softmax')
    ])"""

"""from tensorflow import keras
model = models.Sequential()

model.add(layers.Conv1D(256, 5,padding='same',
                 input_shape=(X_train_expanded_dims.shape[1],1)))
model.add(layers.Activation('relu'))
model.add(layers.Conv1D(128, 5,padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(128, 5,padding='same',))
model.add(layers.Activation('relu'))
model.add(layers.Conv1D(128, 5,padding='same',))
model.add(layers.Activation('relu'))
model.add(layers.Conv1D(128, 5,padding='same',))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(128, 5,padding='same',))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())
model.add(layers.Dense(8))
model.add(layers.Activation('softmax'))
opt = keras.optimizers.Adam(lr=0.00001, decay=1e-6)"""

"""model = models.Sequential()
model.add(layers.Conv1D(256, 8, padding="same", input_shape=(X_train_expanded_dims.shape[1],1), activation = "tanh"))
model.add(layers.Conv1D(256, 8, padding='same', activation = "relu"))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation = "softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])"""

'model = models.Sequential()\nmodel.add(layers.Conv1D(256, 8, padding="same", input_shape=(X_train_expanded_dims.shape[1],1), activation = "tanh"))\nmodel.add(layers.Conv1D(256, 8, padding=\'same\', activation = "relu"))\n#model.add(layers.BatchNormalization())\nmodel.add(layers.Dropout(0.25))\nmodel.add(layers.MaxPooling1D(pool_size=(8)))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\n#model.add(layers.BatchNormalization())\nmodel.add(layers.Dropout(0.25))\nmodel.add(layers.MaxPooling1D(pool_size=(8)))\nmodel.add(layers.Conv1D(64, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(64, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Flatten())\nmodel.add(layers.Dense(8, activation = "softmax"))\nmodel.compile(loss=

In [20]:
#Good model
"""from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow import keras
model = models.Sequential()
model.add(layers.Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(layers.Conv1D(256, 8, padding='same'))
model.add(layers.Activation('relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
#model.add(layers.Conv1D(128, 8, padding='same'))#
#model.add(layers.Conv1D(128, 8, padding='same'))#
#model.add(layers.Activation('relu'))#
#model.add(layers.Conv1D(128, 8, padding='same'))#
#model.add(layers.Activation('relu'))#
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(64, 8, padding='same'))
model.add(layers.Activation('relu'))
#model.add(layers.Conv1D(64, 8, padding='same'))
#model.add(layers.Activation('relu'))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation = "softmax"))
#opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
opt = keras.optimizers.Adam(lr=0.0001)
#opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
model.summary()"""
#0.36"""

'from keras.layers import Dense, Embedding, LSTM\nfrom keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization\nfrom keras.layers import Conv1D, MaxPooling1D, AveragePooling1D\nfrom tensorflow import keras\nmodel = models.Sequential()\nmodel.add(layers.Conv1D(256, 8, padding=\'same\',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns\nmodel.add(Activation(\'relu\'))\nmodel.add(layers.Conv1D(256, 8, padding=\'same\'))\nmodel.add(layers.Activation(\'relu\'))\n#model.add(layers.BatchNormalization())\nmodel.add(layers.Dropout(0.25))\nmodel.add(layers.MaxPooling1D(pool_size=(8)))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\n#model.add(layers.Conv1D(128, 8, padding=\'same\'))#\n#model.add(layers.Conv1D(128, 

In [21]:
"""model = models.Sequential()
model.add(layers.Conv1D(256, 8, padding="same", input_shape=(X_train_expanded_dims.shape[1],1), activation = "tanh"))
model.add(layers.Conv1D(256, 8, padding='same', activation = "relu"))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation = "softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
opt = keras.optimizers.Adam(lr=0.00001, decay=1e-6)"""

'model = models.Sequential()\nmodel.add(layers.Conv1D(256, 8, padding="same", input_shape=(X_train_expanded_dims.shape[1],1), activation = "tanh"))\nmodel.add(layers.Conv1D(256, 8, padding=\'same\', activation = "relu"))\n#model.add(layers.BatchNormalization())\nmodel.add(layers.Dropout(0.1))\nmodel.add(layers.MaxPooling1D(pool_size=(8)))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(128, 8, padding=\'same\', activation = "relu"))\n#model.add(layers.BatchNormalization())\nmodel.add(layers.Dropout(0.1))\nmodel.add(layers.MaxPooling1D(pool_size=(8)))\nmodel.add(layers.Conv1D(64, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Conv1D(64, 8, padding=\'same\', activation = "relu"))\nmodel.add(layers.Flatten())\nmodel.add(layers.Dense(8, activation = "softmax"))\nmodel.compile(loss="c

In [22]:
"""import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
#from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow import keras

model = Sequential()

model.add(layers.Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(layers.Conv1D(256, 8, padding='same'))
model.add(layers.Activation('relu'))

model.add(Conv1D(128, 5,padding='same',input_shape=(X_train_expanded_dims.shape[1],1)))      #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)"""

"import keras\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport tensorflow as tf\nfrom keras.preprocessing import sequence\nfrom keras.models import Sequential\nfrom keras.layers import Dense, Embedding\n#from keras.utils import to_categorical\nfrom keras.layers import Input, Flatten, Dropout, Activation\nfrom keras.layers import Conv1D, MaxPooling1D\nfrom keras.models import Model\nfrom keras.callbacks import ModelCheckpoint\nfrom tensorflow import keras\n\nmodel = Sequential()\n\nmodel.add(layers.Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns\nmodel.add(Activation('relu'))\nmodel.add(layers.Conv1D(256, 8, padding='same'))\nmodel.add(layers.Activation('relu'))\n\nmodel.add(Conv1D(128, 5,padding='same',input_shape=(X_train_expanded_dims.shape[1],1)))      #1\nmodel.add(Activation('relu'))\nmodel.add(Dropout(0.1))\nmodel.add(MaxPooling1D(pool_size=(8)))\n\n\n\nmodel.add(Conv1D(128, 5,padding='same',))                     

In [23]:
"""from tensorflow import keras
model=models.Sequential()
###first layer
model.add(layers.Dense(100,input_shape=(X_train.shape[1],1)))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))
###second layer
model.add(layers.Dense(200))
model.add(layers.Activation('relu'))
###third layer
model.add(layers.Dense(100))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())

model.add(layers.Dense(200))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())

model.add(layers.Dense(100))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())





###final layer
model.add(layers.Dense(8))
model.add(layers.Activation('softmax'))
opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)"""
#quick 0.3

"from tensorflow import keras\nmodel=models.Sequential()\n###first layer\nmodel.add(layers.Dense(100,input_shape=(X_train.shape[1],1)))\nmodel.add(layers.Activation('relu'))\nmodel.add(layers.Dropout(0.25))\n###second layer\nmodel.add(layers.Dense(200))\nmodel.add(layers.Activation('relu'))\n###third layer\nmodel.add(layers.Dense(100))\nmodel.add(layers.Activation('relu'))\nmodel.add(layers.Flatten())\n\nmodel.add(layers.Dense(200))\nmodel.add(layers.Activation('relu'))\nmodel.add(layers.Flatten())\n\nmodel.add(layers.Dense(100))\nmodel.add(layers.Activation('relu'))\nmodel.add(layers.Dropout(0.25))\nmodel.add(layers.Flatten())\n\n\n\n\n\n###final layer\nmodel.add(layers.Dense(8))\nmodel.add(layers.Activation('softmax'))\nopt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)"

In [26]:
from tensorflow import keras
model=models.Sequential()
model.add(layers.Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(layers.Activation('relu'))
model.add(layers.Conv1D(256, 8, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.25))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
#model.add(layers.Conv1D(128, 8, padding='same'))#
#model.add(layers.Conv1D(128, 8, padding='same'))#
#model.add(layers.Activation('relu'))#
#model.add(layers.Conv1D(128, 8, padding='same'))#
#model.add(layers.Activation('relu'))#
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(64, 8, padding='same'))
model.add(layers.Activation('relu'))
#model.add(layers.Conv1D(64, 8, padding='same'))
#model.add(layers.Activation('relu'))
model.add(layers.Dense(100,input_shape=(X_train.shape[1],1)))
model.add(layers.Activation('relu'))
###second layer
model.add(layers.Dense(200))
model.add(layers.Activation('relu'))
###third layer
model.add(layers.Dense(100))
model.add(layers.Activation('relu'))

model.add(layers.Dense(200))
model.add(layers.Activation('relu'))

model.add(layers.Dense(100))
model.add(layers.Activation('relu'))


model.add(layers.Flatten())
model.add(layers.Dense(8, activation = "softmax"))
opt = keras.optimizers.Adam(lr=0.0001)
model.summary()
###first layer


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 229, 256)          2304      
                                                                 
 activation_5 (Activation)   (None, 229, 256)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 229, 256)          524544    
                                                                 
 activation_6 (Activation)   (None, 229, 256)          0         
                                                                 
 dropout_2 (Dropout)         (None, 229, 256)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 28, 256)          0         
 )                                                               
                                                      

/Users/guilhermecavalcantidesabarreto/.pyenv/versions/3.8.12/envs/silvertone/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [27]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train_expanded_dims, y_train_array, validation_split=0.3, epochs=500, batch_size=32, callbacks=[es])


Epoch 1/500
26/26 [==============================] - 15s 541ms/step - loss: 2.0782 - accuracy: 0.1241 - val_loss: 2.0760 - val_accuracy: 0.1676
Epoch 2/500
26/26 [==============================] - 14s 533ms/step - loss: 2.0672 - accuracy: 0.1253 - val_loss: 2.0616 - val_accuracy: 0.1618
Epoch 3/500
26/26 [==============================] - 14s 535ms/step - loss: 2.0557 - accuracy: 0.1340 - val_loss: 2.0498 - val_accuracy: 0.2110
Epoch 4/500
26/26 [==============================] - 14s 534ms/step - loss: 2.0306 - accuracy: 0.1687 - val_loss: 2.0050 - val_accuracy: 0.1965
Epoch 5/500
26/26 [==============================] - 14s 536ms/step - loss: 2.0133 - accuracy: 0.1985 - val_loss: 1.9830 - val_accuracy: 0.2457
Epoch 6/500
26/26 [==============================] - 14s 536ms/step - loss: 1.9963 - accuracy: 0.1973 - val_loss: 2.0068 - val_accuracy: 0.2081
Epoch 7/500
26/26 [==============================] - 14s 553ms/step - loss: 1.9574 - accuracy: 0.2357 - val_loss: 1.9290 - val_accuracy:

In [25]:
#optimiser = keras.optimizers.Adam(learning_rate=0.0001)
#model.compile(optimizer=optimiser,loss='sparse_categorical_crossentropy',metrics=["accuracy"])

In [26]:
#model.summary()

In [27]:
#es = EarlyStopping(patience=10, restore_best_weights=True)
#model.fit(X_train_expanded_dims, y_train_array, validation_split=0.3, epochs=500, batch_size=32, callbacks=[es])

In [28]:
model.evaluate(X_test_expanded_dims, y_test_array)

9/9 [==============================] - 1s 75ms/step - loss: 1.8147 - accuracy: 0.3125


[1.8147146701812744, 0.3125]